# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `01_materials/labs/2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [74]:
%load_ext dotenv
%dotenv 


The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [75]:
import dask.dataframe as dd


+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [76]:
import os
from glob import glob

PRICE_DATA = os.getenv("PRICE_DATA") 
#parquet_files = glob(PRICE_DATA+"/**/*.parquet/*.parquet")
parquet_files = glob(os.path.join(os.getenv('PRICE_DATA'), "**", "*", "*.parquet"))
df = dd.read_parquet(parquet_files)
df



,Date,Open,High,Low,Close,Adj Close,Volume,source,ticker,Year
npartitions=3015,,,,,,,,,,
,datetime64[ns],float64,float64,float64,float64,float64,int64,string,string,int32
,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...


In [77]:
from glob import glob
import os
import pandas as pd
import random
import dask.dataframe as dd
import dask

In [78]:
#os.get('PRICE_DATA')
#glob(os.path.join(os.getenv('PRICE_DATA')))
#PRICE_DATA = os.getenv("PRICE_DATA")


In [79]:
# glob(os.path.join(os.getenv('PRICE_DATA'), "**", "*.parquet"), recursive=True)
parquet_files = glob(os.path.join(os.getenv('PRICE_DATA'), "**", "*.parquet"), recursive=True)
parquet_files

['../../05_src/data/prices\\ABBV\\ABBV_2013\\part.0.parquet',
 '../../05_src/data/prices\\ABBV\\ABBV_2013\\part.1.parquet',
 '../../05_src/data/prices\\ABBV\\ABBV_2014\\part.0.parquet',
 '../../05_src/data/prices\\ABBV\\ABBV_2014\\part.1.parquet',
 '../../05_src/data/prices\\ABBV\\ABBV_2015\\part.0.parquet',
 '../../05_src/data/prices\\ABBV\\ABBV_2015\\part.1.parquet',
 '../../05_src/data/prices\\ABBV\\ABBV_2016\\part.0.parquet',
 '../../05_src/data/prices\\ABBV\\ABBV_2016\\part.1.parquet',
 '../../05_src/data/prices\\ABBV\\ABBV_2017\\part.0.parquet',
 '../../05_src/data/prices\\ABBV\\ABBV_2017\\part.1.parquet',
 '../../05_src/data/prices\\ABBV\\ABBV_2018\\part.0.parquet',
 '../../05_src/data/prices\\ABBV\\ABBV_2018\\part.1.parquet',
 '../../05_src/data/prices\\ABBV\\ABBV_2019\\part.0.parquet',
 '../../05_src/data/prices\\ABBV\\ABBV_2019\\part.1.parquet',
 '../../05_src/data/prices\\ABBV\\ABBV_2020\\part.0.parquet',
 '../../05_src/data/prices\\ABBV\\ABBV_2020\\part.1.parquet',
 '../../

In [80]:
random.seed(42)
n_sample = 60
paraquet_files = random.sample(parquet_files, n_sample)


In [81]:
# Read Dask data
dd_px = dd.read_parquet(paraquet_files).set_index("ticker")
#dd_prices = dd.read_parquet(paraquet_files)
dd_px

,Date,Open,High,Low,Close,Adj Close,Volume,source,Year
npartitions=38,,,,,,,,,
ACN,datetime64[ns],float64,float64,float64,float64,float64,float64,string,int32
ALL,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...
ZIXI,...,...,...,...,...,...,...,...,...
ZIXI,...,...,...,...,...,...,...,...,...


In [82]:
len(dd_px), len(paraquet_files)


(6977, 60)

In [83]:
#feature- for each ticker and

#add_feat = dd_price.assign(
 #    Close_lag_1 = dd_price.groupby("ticker")["Close"].shift(1),
  #   Adj_Close_lag_1 =dd_price.groupby("ticker")["Adj Close"].shift(1),
    # returns=lambda df: (df["Close"] / df.groupby("ticker")["Close"].shift(1)) 
     #hi_lo_range=lambda df: (df["High"] - df["Low"]) / df["Low"] #high and low price of ticker

In [84]:
dd_feat = (
    dd_px
        .groupby("ticker", group_keys=False)
        .apply(
            lambda x: (
                x.sort_values("Date")
                       .assign(
                           Close_lag_1=x["Close"].shift(1), 
                           Adj_Close_lag_1=x["Adj Close"].shift(1),
                           returns=lambda df: (df["Close"] / df["Close_lag_1"]) - 1,
                           hi_lo_range=lambda df: df["High"] - df["Low"]
                )
            ),      
            meta = pd.DataFrame(
                   data ={
                        "Date": "datetimes[ns]",
                        "Open": 'f8',
                        "High": 'f8',
                        "Low": 'f8',
                        "Close": 'f8',
                        "Adj Close": 'f8',
                        "Volume": 'i8',
                        "source": 'object',
                        "Year": 'int32',
                        "Close_lag_1": 'f8',
                        "Adj_Close_lag_1": 'f8',
                        "returns": 'f8',
                        "hi_lo_range": 'f8'
                    },
                    index = pd.Index([], dtype=pd.StringDtype(), name='ticker')
            )
        )
)

In [85]:
dd_feat

,Date,Open,High,Low,Close,Adj Close,Volume,source,Year,Close_lag_1,Adj_Close_lag_1,returns,hi_lo_range
npartitions=38,,,,,,,,,,,,,
ACN,object,object,object,object,object,object,object,object,object,object,object,object,object
ALL,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZIXI,...,...,...,...,...,...,...,...,...,...,...,...,...
ZIXI,...,...,...,...,...,...,...,...,...,...,...,...,...


For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Close:
    
    - `returns`: (Close / Close_lag_1) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [86]:
# Compute Convert to pandas
df = dd_feat.compute()
df



,Date,Open,High,Low,Close,Adj Close,Volume,source,Year,Close_lag_1,Adj_Close_lag_1,returns,hi_lo_range
ticker,,,,,,,,,,,,,
ACN,2006-01-03,28.840000,29.500000,28.820000,29.299999,22.293606,3843400.0,ACN.csv,2006,NaN,NaN,NaN,0.680000
ACN,2006-01-04,29.379999,29.459999,29.049999,29.459999,22.415350,1953800.0,ACN.csv,2006,29.299999,22.293606,0.005461,0.410000
ACN,2006-01-05,29.480000,29.820000,29.340000,29.680000,22.582743,4119300.0,ACN.csv,2006,29.459999,22.415350,0.007468,0.480000
ACN,2006-01-06,31.000000,31.360001,29.900000,31.150000,23.701221,7378900.0,ACN.csv,2006,29.680000,22.582743,0.049528,1.460001
ACN,2006-01-09,30.799999,31.150000,30.750000,31.059999,23.632746,1904800.0,ACN.csv,2006,31.150000,23.701221,-0.002889,0.400000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZIXI,2007-12-24,5.150000,5.230000,5.050000,5.180000,5.180000,719200.0,ZIXI.csv,2007,5.060000,5.060000,0.023715,0.180000
ZIXI,2007-12-26,5.220000,5.250000,4.900000,5.100000,5.100000,995100.0,ZIXI.csv,2007,5.180000,5.180000,-0.015444,0.350000
ZIXI,2007-12-27,5.120000,5.120000,4.850000,4.850000,4.850000,502300.0,ZIXI.csv,2007,5.100000,5.100000,-0.049020,0.270000


In [87]:
#df.groupby(level='ticker')['returns'].mean()
df.groupby('ticker')['returns'].mean()

ticker
ACN     -0.000131
ALL      0.547300
AMAL    -0.001805
AMT      0.000685
BPYPN   -0.014343
BWEN    -0.002701
BWG     -0.000106
CAH      5.531761
CBB      0.000300
CBT      0.000928
CGEN     0.389819
ESGR     0.002150
ESXB     0.001524
ETJ     -0.000441
GAZ     -0.004759
GEL      0.003475
GLW      0.162547
GURE     0.025304
ITCB     0.005179
LH      -0.000058
MLPY     0.000607
MNK      0.003217
MOS      0.004322
NGD     -0.003628
PBBI     0.003388
PSA      0.000655
QQQX    -0.001368
SCSC     0.000248
SMG      0.001974
SPXC     2.351015
TEF      0.007560
TNC      2.113861
TSN      0.015027
USV      0.000479
VTR      0.001772
WST      0.001894
XENE    -0.012708
ZIXI     0.008774
Name: returns, dtype: float64

In [88]:
# 10-day moving average of returns for each ticker in pandas data frame
df["returns_movingaver"] = (
    df
    .groupby('ticker')['returns']
    .rolling(10)
    .mean()
    .reset_index(level=0, drop=True)
)

In [89]:
df

,Date,Open,High,Low,Close,Adj Close,Volume,source,Year,Close_lag_1,Adj_Close_lag_1,returns,hi_lo_range,returns_movingaver
ticker,,,,,,,,,,,,,,
ACN,2006-01-03,28.840000,29.500000,28.820000,29.299999,22.293606,3843400.0,ACN.csv,2006,NaN,NaN,NaN,0.680000,NaN
ACN,2006-01-04,29.379999,29.459999,29.049999,29.459999,22.415350,1953800.0,ACN.csv,2006,29.299999,22.293606,0.005461,0.410000,NaN
ACN,2006-01-05,29.480000,29.820000,29.340000,29.680000,22.582743,4119300.0,ACN.csv,2006,29.459999,22.415350,0.007468,0.480000,NaN
ACN,2006-01-06,31.000000,31.360001,29.900000,31.150000,23.701221,7378900.0,ACN.csv,2006,29.680000,22.582743,0.049528,1.460001,NaN
ACN,2006-01-09,30.799999,31.150000,30.750000,31.059999,23.632746,1904800.0,ACN.csv,2006,31.150000,23.701221,-0.002889,0.400000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZIXI,2007-12-24,5.150000,5.230000,5.050000,5.180000,5.180000,719200.0,ZIXI.csv,2007,5.060000,5.060000,0.023715,0.180000,-0.006491
ZIXI,2007-12-26,5.220000,5.250000,4.900000,5.100000,5.100000,995100.0,ZIXI.csv,2007,5.180000,5.180000,-0.015444,0.350000,0.005770
ZIXI,2007-12-27,5.120000,5.120000,4.850000,4.850000,4.850000,502300.0,ZIXI.csv,2007,5.100000,5.100000,-0.049020,0.270000,-0.004882


+ Convert the Dask data frame to a pandas data frame. 
+ Add a new feature containing the moving average of `returns` using a window of 10 days. There are several ways to solve this task, a simple one uses `.rolling(10).mean()`.

(3 pt)

Please comment:

Converting to pandas is not necessary but it is often clean, fast and convenient. Dash usually allow rolling window operations such as .mean(), .groupby(), .rolling(). Dask is used for very large dataset. It would still have been possible to compute the moving average of returns in Dash without having to convert it to pandas.

Writing it in Dash would have been better because Dash is better for big dataset and Dash can handle  dataset that does not fit in memory because it will be stored in disk.  It does calculations in parallel and out-of-core. Converting to pandals entails loading the data into memory wich can be slow or even impossible for big dataset. 



+ Was it necessary to convert to pandas to calculate the moving average return?
+ Would it have been better to do it in Dask? Why?

(1 pt)

## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.